# Hello Bluesky: 'Counting' a detector

## Configuration

This code would normally go in a script automatically run at startup. The user would not have to worry about this.

In [0]:
%matplotlib notebook

from bluesky import RunEngine
from bluesky.plans import count
from bluesky.callbacks import LiveTable, LivePlot

# Make plots live-update while scans run.
from bluesky.utils import install_nb_kicker
install_nb_kicker()

# Set up simulated hardware.
from bluesky.examples import motor, SynGauss
det = SynGauss('det', motor, 'motor', center=0, Imax=1,
               noise='uniform', sigma=1, noise_multiplier=0.1)

RE = RunEngine({})
table = LiveTable(['det'])

## Data Acquisition

### Executing a `count` plan with various parameters

In [0]:
RE(count([det]), table)

The return value is a list of the run IDs that uniquely identify this data set. The "scan num" is easier to remember but is not good for long-term reference because it may not be unique.

In [0]:
count?

In [0]:
# five consecutive readings
RE(count([det], num=5), table)

In [0]:
# add a plot (not a very exciting plot...)
RE(count([det], num=5), [table, LivePlot('det')])

In [0]:
# five sequential readings separated by a 1-second delay
RE(count([det], num=5, delay=1), table)

In [0]:
# a variable delay
RE(count([det], num=5, delay=[1, 2, 3, 4]), table)

In [0]:
# multiple detectors
from bluesky.examples import det1, det2
bigger_table = LiveTable(['det1', 'det2'])

RE(count([det1, det2], num=3), bigger_table)

### Composing plans

In [0]:
def multi_count(detectors):
    "Three runs with 1, 2, 3 exposures each."
    for i in [1, 2, 3]:
        yield from count(detectors, num=i)

In [0]:
RE(multi_count([det]), table)

Notice that `count` is invoked three times, and that each invocation generates a new scan number and run ID.

## Exercises

1. Count a detector 3 times sequentially, generating one run.
2. Count a detector 3 times sequentially, generating a separate run (i.e., a separate 'scan num') for each reading.
3. Count a detector with exponentially increasing delays between readings.